## NYPD Data

### Imports

In [ ]:
# data import
import pickle

# data manipulation
import numpy as np
import pandas as pd
from datetime import datetime

# plotting
import matplotlib.pyplot as plt

#gaussian filter
from scipy.ndimage import gaussian_filter

from minority_report.matrix import Matrix

### Data Import

In [ ]:
pickle_path = ('../raw_data/clean.pickle')
with open(pickle_path, 'rb') as f:
    df = pickle.load(f)

### Sample Selection

In [ ]:
inf = df['period'] >= datetime(2017, 1, 1, 0, 0, 0)
sup = df['period'] < datetime(2017, 3, 1, 0, 0, 0)
sample = df[ inf & sup ]

In [ ]:
sample = sample[sample['precinct_number']==75]

In [ ]:
sample.shape

In [ ]:
sample['longitude'].min(), sample['longitude'].max(), sample['latitude'].min(), sample['latitude'].max() 

In [ ]:
del df

In [ ]:
sample.groupby('period').count().head()

In [ ]:
np.sort(sample['period'].unique())

### DataGenerator

In [ ]:
import tensorflow as tf
​
class CustomDataGenerator(tf.keras.utils.Sequence):
  def __init__(self, df, batch_size=32, shuffle=True, lat_meters=100, lon_meters=100, *args, **kwargs):
    self.df = df
    self.batch_size = batch_size
    self.indices = np.arange(len(self.df))
    self.shuffle = shuffle
    self.lat_meters = lat_meters
    self.lon_meters = lon_meters
    self.on_epoch_end()
​
  def __len__(self):
    """
    Return the number of batches so the generator knows how many batches
    it should generate for each epoch
    """
    return len(df) // self.batch_size
​
  def __getitem__(self, i):
    """
    Return the i-th batch
    """
    batch_indices = self.indices[i * self.batch_size:(i + 1) * self.batch_size]
    batch = self.df.iloc[batch_indices] 
    return self.__get_data(batch)
​
  def __get_data(self, batch):
    X = from_coord_to_matrix(batch, lat_meters=self.lat_meters, lon_meters=self.lon_meters)
    y = from_coord_to_matrix(batch, lat_meters=self.lat_meters, lon_meters=self.lon_meters) # à changer
    return X, y
​
  def on_epoch_end(self):
    self.indices = np.arange(len(self.df))
    if self.shuffle == True:
        np.random.shuffle(self.indices)

In [ ]:
BATCH_SIZE = 2
​
datagen = CustomDataGenerator(df, batch_size=BATCH_SIZE)
p, q = datagen.__getitem__(3)
print(p.shape)
print(q.shape)

### Lat - Long to Array

In [ ]:
def from_meters_to_coords(lat_meters, lon_meters):
    """
    gives the latitude and longitude step to use for the grid buckets
    lat_meters, lon_meters = lat/lon step
    """
    #Position, decimal degrees
    lat = 40
    lon = -73

    #Earth’s radius, sphere
    R=6378137

    #offsets in meters
    dn = lat_meters
    de = lon_meters

    #Coordinate offsets in radians
    dLat = dn/R
    dLon = de/(R*np.cos(np.pi*lat/180))

    #OffsetPosition, decimal degrees
    latO = dLat * 180/np.pi
    lonO = dLon * 180/np.pi 

    return latO, lonO

In [ ]:
def from_coord_to_matrix(df, lat_meters, lon_meters):
    """
    outputs the 3D matrix of all coordinates for a given bucket height and width in meters
    """
    df=df.copy()
    #add 'time_index' column to df
    ind = {time:index for index,time in enumerate(np.sort(df['period'].unique()))}
    df['time_index'] = df['period'].map(ind)
    
    #print(df.groupby('time_index').count())
    
    #initiate matrix
    #40.49611539518921, 40.91553277600008, -74.25559136315213,-73.70000906387347) : NYC boundaries
    #([40.56952999448672, 40.73912795313436],[-74.04189660705046, -73.83355923946421]) : brooklyn boundaries
    #[40.6218192717505, 40.6951504231971],[-73.90404639808888, -73.83559344190869]) :precinct 75 boundaries
    
    grid_offset = np.array([ -sample['latitude'].max() , sample['longitude'].min(), 0 ]) # Where do you start
    #from meters to lat/lon step
    lat_spacing, lon_spacing = from_meters_to_coords(lat_meters, lon_meters )
    grid_spacing = np.array([lat_spacing , lon_spacing, 1 ]) # What's the space you consider (euclidian here)
    
    
    
    #get points coordinates
    coords = np.array([( -lat, lon,t_ind) for lat, lon,t_ind \
                   in zip(df['latitude'],df['longitude'],df['time_index'])])
    
    
    # Convert point to index
    indexes = np.round((coords - grid_offset)/grid_spacing).astype('int')
    X = indexes[:,0] 
    Y = indexes[:,1] 
    Z = indexes[:,2] 
    #print(len(Z[Z==0]))
    
    #print(X.min(), X.max(), Y.min(), Y.max(),Z.min(),Z.max())
    
    #virgin matrix
    a = np.zeros((X.max()+1, Y.max()+1, Z.max()+1))
   
    
    a[X, Y, Z]=1
    
    return a, a.shape[1], a.shape[2]

In [ ]:
img, lat_size, lon_size = from_coord_to_matrix(sample, 10, 8)

In [ ]:
img.shape

In [ ]:
img[:,:,0].sum()

In [ ]:
plt.figure(figsize=(10, 10))

plt.imshow(img[:,:,0], cmap='gray');

### Plotting Array (with and w/o Gaussian filter)

In [ ]:
#plt.figure(figsize=(5, 5))
#plt.imshow(img[3], cmap='gray');

In [ ]:
img3D_conv = gaussian_filter(img, sigma=(3, 3, 6))

In [ ]:
img3D_conv.shape

In [ ]:
plt.figure(figsize=(10, 10))

plt.imshow(img3D_conv[:,:,0], cmap='gray', vmax = img3D_conv[:,:,0].max());

### Splitting into X and y

#### Try outs

In [ ]:
img3D_conv.shape

In [ ]:
img3D_conv.max()

In [ ]:
obs_lon = 2 # 2*10m together in lon
obs_lat = 3 # 3*10m
obs_time = 6 # 6h stacked together
obs_tf = 24 # 24h en tout
tar_tf = 12 # 12h en tout pour target

In [ ]:
length = obs_tf + tar_tf
position = np.random.randint(0, img3D_conv.shape[2] - length)

In [ ]:
position

In [ ]:
subsample = img3D_conv[:,:,position:position + length]

In [ ]:
subsample.shape

In [ ]:
subsample.max()

In [ ]:
observations, targets = np.split(subsample,[obs_tf], axis=2)

In [ ]:
observations.shape

In [ ]:
observations.max()

#### Math test

In [ ]:
a = np.array([[[1, 2, 3], [4, 5, 6]], [[1.1, 2.2, 3.3], [4.4, 5.5, 6.6]]])
b = np.array([[[7, 8, 9], [10, 11, 12]], [[7.7, 8.8, 9.9], [10.1, 11.1, 12.1]]])
a

In [ ]:
a.shape

In [ ]:
a.reshape((1, 2, 2, 2))

In [ ]:
c += b

In [ ]:
c

#### Resume

In [ ]:
timeframes=[]
start=0
nb_images = int(observations.shape[2]/obs_s)
for i in range (nb_images):
    not_stacked = observations[: , : , start : start+obs_s]
    base = not_stacked[:,:,0]
    for i in range(1,obs_s):
        base += first_timeframe[:,:,i]
    timeframes.append(not_stacked)
    start += obs_s

In [ ]:
tf_arr = np.array(timeframes)
tf_arr.shape

In [ ]:
tf_arr.max()

In [ ]:
first_timeframe = tf_arr[0]
first_timeframe.shape

In [ ]:
first_timeframe.max()

In [ ]:
base = first_timeframe[:,:,0]
base.shape

In [ ]:
base.max()

In [ ]:
for i in range(1,obs_s+1):
    base += first_timeframe[:,:,i]

In [ ]:
base.shape

In [ ]:
base.max()

In [ ]:
print(base)

#### Try with othe grid

In [ ]:
stacked = stacking_2(observations, obs_lat , obs_lon, obs_time )
stacked

In [ ]:
stacked.max()

In [ ]:
observations[observations != 0]

#### Functions

In [ ]:
def stacking(img3D, window, lat_step, lon_step, time_step):
    
    grid_offset = np.array([0,0,0]) # Where do you start
    
    #new steps from precise grid
    grid_spacing = np.array([lat_step , lon_step, time_step]) 
    #get points coordinates
    coords = np.argwhere(window)
    flat = window.flatten()
    values = flat[flat !=0]
    
    # Convert point to index
    indexes = np.round((coords - grid_offset)/grid_spacing).astype('int')
    X = indexes[:,0] 
    Y = indexes[:,1] 
    Z = indexes[:,2]
    
    #virgin matrix
    a = np.zeros((int(img3D.shape[0]/lat_step)+1, int(img3D.shape[1]/lon_step)+1,Z.max()+1))  
    
    for i in range(len(indexes)):
        if a[X[i], Y[i], Z[i]] == 0:
            a[X[i], Y[i], Z[i]] = values[i]
        else:
            a[X[i], Y[i], Z[i]] += values[i]
                        
    return a

In [ ]:
#def stacking(window, step, timeframe):

    #nb_images = int(window.shape[2]/step) #nb of image in one window
    #print(nb_images)
    
#    not_stacked = np.split(window, step, axis=2) #get the group of hours to stack together
#    stacked = np.array([box.sum(axis=2) for box in not_stacked])
    #for element in no_stacked :
    #    stacked = element[:,:,0] #intialise the base on which we are going to stack the others    
    #    for i in range(1,step):
    #        stacked += element[:,:,i] #stack all the other hours of the group onto the base
    #    timeframes.append(stacked) #get stackedhours for all the images of the window

    
#    return stacked

In [ ]:
def get_observation_target(img3D,
                           obs_timeframe,obs_lat,obs_lon, obs_time,
                           target_timeframe,  tar_lat,tar_lon, tar_time):
    '''
    output an observation of x_length consecutive images and the y_length next images as the target
    obs_step, obs_timeframe, target_step, target_timeframe : unit = hours
    '''
    #function from raw to hours
    #print('creating obs')
    length = obs_timeframe + target_timeframe
    
    position = np.random.randint(0, img3D_conv.shape[2] - length)

    subsample = img3D[:, :, position : position + length]
    #print(subsample.shape)
    
    observations, targets = np.split(subsample,[obs_timeframe], axis=2) # divide the subsample in X and y
    
    #print(observations.shape)
    #print(observations.min(), observations.max())
    
    observation = stacking(img3D, observations, obs_lat, obs_lon, obs_time) #get stacked hours for all images
    print(observation.shape)
    #print (targets.shape)
    
    target = stacking(img3D, targets,  tar_lat, tar_lon, tar_time )
    print(target.shape)
    return observation, target

In [ ]:
observation, target = get_observation_target(img3D_conv,
                                             obs_tf, obs_lat,obs_lon,  obs_time,
                                             tar_tf, tar_lat,tar_lon,  tar_time)

In [ ]:
observation.shape

In [ ]:
target.shape

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(observation[:,:,0], cmap=('gray'))

### Subsampling to get multiple X and y

In [ ]:
def get_X_y(img3D_conv, nb_observations, obs_tf,obs_lat,obs_lon, obs_time,
                    tar_tf, tar_lat,tar_lon, tar_time):
        '''
        outputs n observations and their associated targets
        '''
        X = []
        y = []
        for n in range(nb_observations):
            print(f'Creating observation {n+1} out of {nb_observations}')
            X_subsample, y_subsample = get_observation_target(img3D_conv,
                                           obs_tf,obs_lat,obs_lon, obs_time,
                                           tar_tf,  tar_lat,tar_lon, tar_time)
            X.append(X_subsample)
            y.append(y_subsample)

        X = np.array(X)
        y = np.array(y)
        return X, y

In [ ]:
obs_lon = 5 # 5*5m together in lon
obs_lat = 5 # 5*5m
obs_time = 6 # 3h stacked together
obs_tf = 72 # for 48h straight as X
tar_lon =  5# 5*5m together in lon
tar_lat = 5 # 5*5m
tar_time = 6 # 2h stacked together
tar_tf = 12 # for 12h straight as y
nb_observations = 20

X, y = get_X_y(img3D_conv, nb_observations,
               obs_tf,obs_lat,obs_lon, obs_time,
               tar_tf, tar_lat,tar_lon, tar_time)

In [ ]:
del img, img3D_conv, inf, sup, sample

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
with open('../raw_data/X_by_hour_try2.pickle', 'wb') as f:
    pickle.dump(X, f)
with open('../raw_data/y_by_hour_try2.pickle', 'wb') as f:
    pickle.dump(y, f)    

## Model

### Imports

In [2]:
# Train, test, split
from sklearn.model_selection import train_test_split

# Model
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

### Train, test, split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
with open('../raw_data/X_train_by_hour.pickle', 'wb') as f:
    pickle.dump(X_train, f)

In [ ]:
with open('../raw_data/y_train_by_hour.pickle', 'wb') as f:
    pickle.dump(y_train, f)

In [ ]:
with open('../raw_data/X_test_by_hour.pickle', 'wb') as f:
    pickle.dump(X_test, f)

In [ ]:
with open('../raw_data/y_test_by_hour.pickle', 'wb') as f:
    pickle.dump(y_test, f)

In [ ]:
del X,y

### Model

In [3]:
def init_model():
    print('1')
    model = models.Sequential()
    print('1')
    model.add(layers.Conv3D(64, kernel_size = (4,4,4), activation = 'relu', padding='same',
                              input_shape = (138, 99, 15,1)))
    model.add(layers.MaxPooling3D(2))
    
    print('1')
    model.add(layers.Flatten())
    model.add(layers.Dense(56*50*4, activation = 'relu'))
    print('1')
    model.add(layers.Reshape((56, 50, 4)))
    model.compile(loss ='mse',
             optimizer='adam',
             metrics='mae')
    return model

### Running Model (Instance, Early Stopping, Fit and Evaluate)

**Reminders**

**batch_size:** no. of data used to compute error between y_pred and y_true each time weights of neural network updated (use 16 or 32)

**epoch:** once all data has been used once to update the weights

In [ ]:
model = init_model()

1
1
1


In [ ]:
model.summary()

In [ ]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

history = model.fit(X_train, y_train,
                      batch_size = 32, 
                      epochs = 2,
                      validation_split = 0.3,
                      callbacks = es)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['val_loss'])

In [ ]:
plt.plot(history.history['val_mae'])

In [ ]:
print(model.evaluate(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = flatten

In [ ]:
y_pred.reshape(y_length, lat_size, lon_size)

## From coord to map

In [ ]:
def from_matrix_to_coord(matrix, lat_meters, lon_meters):
        """
        gives back the coordinates from a 3D matrix for a given bucket height and width
        """
        results = []
        for observation in matrix:
            # Where do you start
            grid_offset = np.array([0, -40.91553277600008,  -74.25559136315213,])

            #from meters to lat/lon step
            lat_spacing, lon_spacing = from_meters_to_coords(lat_meters, lon_meters)

            # What's the space you consider (euclidian here)
            grid_spacing = np.array([1, lat_spacing, lon_spacing])

            indexes = np.argwhere(observation)
            #print(indexes.shape)
            # index : coords de mes crimes dans mon np array
            result = grid_offset + indexes * grid_spacing
            results.append(result)
        return np.array(results)

In [ ]:
y_pred_notsure = np.where((y_train>5*10**(-4)) & (y_train<5*10**(-3)), y_train, 0)
y_pred_middle = np.where((y_train>5*10**(-3)) & (y_train<1*10**(-2)), y_train, 0)
y_pred_sure = np.where(y_train>1*10**(-2), y_train, 0)

#y_pred_notsure = np.where((y_pred>5*10**(-3)) & (y_pred<1*10**(-2)), y_pred, 0)
#y_pred_middle = np.where((y_pred>1*10**(-2)) & (y_pred<1.8*10**(-2)), y_pred, 0)
#y_pred_sure = np.where(y_pred>1.8*10**(-2), y_pred, 0)

In [ ]:
coords_not_sure = from_matrix_to_coord(y_pred_notsure, 200, 200)
coords_middle = from_matrix_to_coord(y_pred_middle, 200, 200)
coords_sure = from_matrix_to_coord(y_pred_sure, 200, 200)

In [ ]:
coords_back_not_sure_df = pd.DataFrame(coords_not_sure[0], columns=['image', 'lat', 'lon'])
coords_back_not_sure_df['right_lat'] = -coords_back_not_sure_df['lat']

coords_middle_df = pd.DataFrame(coords_middle[0], columns=['image', 'lat', 'lon'])
coords_middle_df['right_lat'] = -coords_middle_df['lat']

coords_back_sure_df = pd.DataFrame(coords_sure[0], columns=['image', 'lat', 'lon'])
coords_back_sure_df['right_lat'] = -coords_back_sure_df['lat']

In [ ]:
last_image_not_sure = coords_back_not_sure_df[coords_back_not_sure_df['image']==2.0]
last_image_middle = coords_middle_df[coords_middle_df['image']==2.0]
last_image_sure = coords_back_sure_df[coords_back_sure_df['image']==2.0]

In [ ]:
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.set_xlim(left=-74.25559136315213, right=-73.70000906387347)
ax.set_ylim(bottom = 40.49611539518921, top=40.91553277600008)
sns.scatterplot(x='lon', y='right_lat', data=last_image_not_sure, s=2, ax=ax)
sns.scatterplot(x='lon', y='right_lat', data=last_image_middle, s=2, ax=ax)
sns.scatterplot(x='lon', y='right_lat', data=last_image_sure, s=2, ax=ax)
plt.legend()

In [ ]:
#modèle clairement pourri car il indique des crimes sur staten island
#voir baseline modèle (on est en dessous)